<a href="https://colab.research.google.com/github/wep13/P1-Software-prodotto-vegani/blob/main/python_project_professioAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BioMarket s.a.s ti assume per sviluppare un piccolo software gestionale per la loro nuova bottega in Via Tan 6.

Il software deve avere le seguenti funzionalità:

1.   Registrare nuovi prodotti, con nome, quantità, prezzo di vendita e prezzo di acquisto.
2.   Elencare tutti i prodotti presenti.
3.   Registrare le vendite effettuate.
4.   Mostrare i profitti lordi e netti.
5.   Mostrare un menu di aiuto con tutti i comandi disponibili.

In [ ]:
#Importing the needed libraries
import csv
import os

In [ ]:
def help():

  """
  The function "help" allows users to visualize all the possible commands within the program.
  """

  help_txt = 'The available commands are the following: \n \
  add: adds a product to the inventory \n \
  inventory_list: shows the list of all the products available in the inventory \n \
  sell: adds a selling for a product \n \
  profit: shows the total profits \n \
  help: shows all the available commands  \n \
  close: closes the program \n'

  return help_txt

In [ ]:
def is_positive():

  """
  The function "is_positive" checks whether the input from the user is a valid positive number.
  """

  while True:
    try:
      value = float(input().replace(",","."))
      if value>=0:
        return value
      else:
        print('The number must be positive!')
    except ValueError:
      print('Please enter a valid positive number.')

In [ ]:
def inventory_list():

  """
  The function "inventory_list" allows users to see all the products in the inventory and their informations
  """

  #We read the csv file and print its content
  with open("inventory.csv","r") as inventory:
    inventory_reader = csv.reader(inventory)
    for row in inventory_reader:
      print(row)


In [ ]:
def add():

  """
  The function "add" allows users to add products to the inventory.
  If the product is already present then the quantity is added to the one already present. In this case there is no option to update the buy and sell price.
  If the product is not present then a new line with all the information is added to the inventory.
  """

  print('What product would you like to add')
  name = input().lower()
  print(f'What quantity of {name} would you like to add?')
  quantity = is_positive()

  #Read the CSV file
  with open("inventory.csv", mode='r') as inventory:
    inventory_dict_r = csv.DictReader(inventory)
    data = []

    #Creating a list of all the row in the inventory
    for row in inventory_dict_r:
      data.append(row)

    #Creating a list of all the item
    item_list = []
    for element in data:
      item_list.append(element['Item'].lower())

    #If the item is already present in the inventory we just need to update his quantity
    if name in item_list:
      index = item_list.index(name)
      data[index]['Quantity'] = float(data[index]['Quantity']) + quantity
      print(f'Item {name} was already present in the inventory. The new quantity has been added.')

    #If the item is not present in the inventory then we need to add a new row
    else:
      print(f'Insert selling price for {name}')
      sell_price = is_positive()
      print(f'Insert buying price for {name}')
      buy_price = is_positive()

      data.append({'Item':name,'Quantity':quantity,'Sell Price':sell_price,'Buy Price':buy_price})
      print(f'ADDED {quantity} X {name}')

    #Now we need to write all the data stored in the list "data" to the file
    with open("inventory.csv", mode='w') as inventory:
      field_names=['Item','Quantity','Sell Price','Buy Price']
      inventory_dict_w = csv.DictWriter(inventory, fieldnames=field_names)
      inventory_dict_w.writeheader()

      for line in data:
        inventory_dict_w.writerow(line)


In [ ]:
def sell():

  """
  The function "sell" allows users to add products to the inventory.
  If the product is not present or if the quantity is not enough then a message error is returned.
  The total profit gross and net are then stored in a separate file.
  The quantity sold is not subtracted to the ones in the inventory.
  """

  #Read the CSV file
  with open('inventory.csv', mode='r') as inventory:
    inventory_dict_r = csv.DictReader(inventory)
    data = []

    #Creating a list of all the row in the inventory
    for row in inventory_dict_r:
      data.append(row)

    #Creating a list of all the item
    item_list = []
    for element in data:
      item_list.append(element['Item'].lower())

    #We ask the user what product was sold and we ensure it is in the inventory
    print('What product was sold?')
    name = input().lower()

    assert name in item_list, f'{name} is not in the inventory'

    #We ask the user which quantity of the product was sold and we ensure it is available in the inventory
    print(f'What quantity of {name} was sold?')
    quantity = is_positive()

    index = item_list.index(name)
    inventory_quantity = float(data[index]['Quantity'])

    assert quantity <= inventory_quantity, f'The max quantity of {name} available is {inventory_quantity}'

    #Now we search for the price of the item in the inventory and we print the information
    sell_price = float(data[index]['Sell Price'])
    gross_profit = quantity*sell_price
    buy_price = float(data[index]['Buy Price'])
    net_profit = gross_profit - quantity*buy_price

    print('SALE REGISTERED')
    print(f'{quantity} X {name} : €{sell_price}')
    print(f'Total: € {gross_profit}')

    #Now we have to update the profit file
    with open('profits.csv', mode='a') as profit_file:
      field_names = ['Gross Profit', 'Net Profit']
      profit_dict = csv.DictWriter(profit_file, fieldnames=field_names)

      #Check if the file is empty to write the header
      profit_file.seek(0, 2)  #Move to the end of the file
      if profit_file.tell() == 0:  #If file is empty
        profit_dict.writeheader()

      profit_dict.writerow({'Gross Profit':gross_profit, 'Net Profit':net_profit})


In [ ]:
def profit():

  """
  The function "profit" is going to return the gross and net profit for the sales.
  The function return the gross profit as the first element and net profit as the second element.
  """
  #Read the CSV file

  with open('profits.csv', mode='r') as profit_file:
    profit_reader = csv.reader(profit_file)

    #Inizialize the list for gross and net profit
    gross_profit = []
    net_profit = []

    #Loop to add element to the lists except the header
    for index,row in enumerate(profit_reader):
      if index >0:
        gross_profit.append(float(row[0]))
        net_profit.append(float(row[1]))

    gross_profit = round(sum(gross_profit),2)
    net_profit = round(sum(net_profit),2)

    return (gross_profit, net_profit)



In [ ]:
print('Welcome to the software of BioMarket s.a.s')
print('Digit a command to execute an action. To see all the possible commands try with "help" \n')

#Verify if the inventory file is present in the directory, if not create it

if os.path.isfile("inventory.csv"):
  pass

else:
  with open("inventory.csv", mode='w', newline='') as inventory:
    inventory.write("Item,Quantity,Sell Price,Buy Price\n")

commands_list = ['add','inventory_list','sell','profit','help','close']

while True:
  command = input('Insert a command: ')

  assert command in commands_list, ('Invalid command. \n'+help())

  if command == 'help':
    print(help())

  elif command =='inventory_list':
    inventory_list()

  elif command =='add':
    add()

  elif command =='sell':
    sell()

  elif command =='profit':
    gross_profit, net_profit = profit()
    print('Gross Profit:', gross_profit)
    print('Net Profit:', net_profit)

  elif command =='close':
    print('Closing the program')
    break